In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from selenium import webdriver

In [2]:
items = []

In [3]:
class SmartFitSpider(scrapy.Spider):
    name = 'smart-fit'
    start_urls = ['https://www.smartfit.com.br/unidades']

    def parse(self, response):
        locations = response.css('div.Locations__item')
        
        for location in locations:
            item = {}
            unit = location.css('h3 > a::text').extract_first()
            item['Unidade'] = unit
            
            plans = location.css('div.Locations__plans__item')
            for plan in plans:
                plan_name = plan.css('h4::text').extract_first()
                integer = plan.css('div.component_location_price__current > div.component_location_price__integer::text').extract_first()
                decimal = plan.css('div.component_location_price__current > div.component_location_price__decimal::text').extract_first()
                current_price = integer + ',' + decimal
                
                item[plan_name] = current_price

            full_address = location.css('div.Locations__content__wrapper > p.Text::text').extract()
            
            address, neighborhood = full_address[0].split('-')
            address, neighborhood = address[:-1], neighborhood[1:]
            
            city_state, code = full_address[-1].split('-')
            city_state, code = city_state[:-1], code[1:]
            
            city, state = city_state.split(',')
            state = state[1:]
            
            item['Endereco'] = address
            item['Bairro'] = neighborhood
            item['Cidade'] = city
            item['Estado'] = state
            item['CEP'] = code
            
            items.append(item)

In [4]:
process = CrawlerProcess()
process.crawl(SmartFitSpider)
process.start()

2019-07-18 17:04:38 [scrapy.utils.log] INFO: Scrapy 1.5.2 started (bot: scrapybot)
2019-07-18 17:04:38 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 19.2.0, Python 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.6.1, Platform Windows-10-10.0.17134-SP0
2019-07-18 17:04:38 [scrapy.crawler] INFO: Overridden settings: {}
2019-07-18 17:04:38 [scrapy.extensions.telnet] INFO: Telnet Password: 3871d62ec27c3de4
2019-07-18 17:04:38 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2019-07-18 17:04:39 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:51116/session {"capabilities": {"firstMatch": [{}], "alwaysMatch": {"browserName": "chrome", "platformName": "any", "goog:chromeOptions": {"extensions": [

In [5]:
items

[]